Question 1: Complex calcium oscillations

In [1]:
# Declare parameters
#---------------------------------------
#Total concentrations (micro Moles)
Ca_tot =  90 
Pr_tot = 120

# Geometric Parameters

rho_ER = 0.01
rho_m = 0.01
beta_ER = 0.0025
beta_m = 0.0025

#Kinetics parameters

k_pump = 20 #s^-1
k_ch = 3900 #s^-1
k_leak = 0.05 #s^-1
k_in = 300 #(micro Moles s^-1)
k_out = 125 #s^-1
k_m = 0.00625 #s^-1
k_plus = 0.1 #(micro Moles)
k_minus = 0.01 #s^-1
K1 = 5 #(micro Moles)
K2 = 0.8 #(micro Moles)
K3 = 5 #(micro Moles)

#Initial concentrations (micro Moles)

Ca_cyt_0 = 0.3 
Ca_ER_0 = 0.2 
Ca_m_0 = 1 

In [2]:
def Ca_system(y, t):
    Ca_cyt, Ca_ER, Ca_m = y
    J_pump = k_pump*Ca_cyt
    J_ch = k_ch*((Ca_cyt**2)/(K1**2 + Ca_cyt**2))*(Ca_ER - Ca_cyt)
    J_leak = k_leak*(Ca_ER - Ca_cyt)
    J_in = k_in*(Ca_cyt**8)/(K2**8 + Ca_cyt**8)
    J_out = Ca_m*(((k_out*Ca_cyt**2) / (K3**2 + Ca_cyt**2))+k_m)
    CaPr = Ca_tot - Ca_cyt -(rho_ER/beta_ER)*Ca_ER - (rho_m/beta_m)*Ca_m
    Pr = Pr_tot - CaPr
    
    return [J_ch + J_leak - J_pump + J_out - J_in + k_minus*CaPr - k_plus*Ca_cyt*Pr, 
           (beta_ER/rho_ER)*(J_pump - J_ch - J_leak), (beta_m/rho_m)*(J_in - J_out)]

Question 2: Schnakenberg model

In [4]:
#Laplace 1D

def laplace_1D(X, dx):
    Xleft = X[:-2]
    Xright = X[2:]
    return (Xleft + Xright - 2*X[1:-1])/dx**2

In [5]:
#Laplace 2D

def laplace_2D(X, dx):
    Xtop = X[:-2, 1:-1]
    Xbottom = X[2:, 1:-1]
    Xleft = X[1:-1, :-2]
    Xright = X[1:-1, 2:]
    return (Xtop + Xbottom + Xleft + Xright - 4*X[1:-1, 1:-1])/(dx**2)